In [1]:
import torch
import MiniFL as mfl

In [2]:
def run_client(i:int, client: mfl.algorithms.gd.Client):
    for _ in range(10):
        loss = client.send_grad_get_loss()
        # print(f"Client {i}: {loss}")
        client.apply_global_step()
        
def run_master(master: mfl.algorithms.gd.Master):
    for i in range(10):
        print(f"Master: {master.round()}")

In [3]:
from sklearn.datasets import load_svmlight_file

NUM_CLIENTS = 20

data, labels = load_svmlight_file("phishing.txt")
enc_labels = labels.copy()
data_dense = data.todense()

eval_data = (torch.from_numpy(data_dense).to(torch.float32), torch.from_numpy(enc_labels).to(torch.float32)[:, None])
clients_data = [(x, y) for x, y in zip(torch.split(eval_data[0], len(eval_data[0]) // NUM_CLIENTS, dim=0), torch.split(eval_data[1], len(eval_data[1]) // NUM_CLIENTS, dim=0))]

model = torch.nn.Linear(eval_data[0].shape[1], 1, bias=False)
loss_fn = torch.nn.BCEWithLogitsLoss()

master, clients = mfl.algorithms.gd.get_master_and_clients(
    lr=200,
    clients_data=clients_data,
    eval_data=eval_data,
    model=model,
    loss_fn=loss_fn,
)

In [4]:
import threading

client_threads = []
for i, client in enumerate(clients):
    client_threads.append(threading.Thread(target=run_client, args=(i, client)))
    client_threads[-1].start()
    
master_thread = threading.Thread(target=run_master, args=(master,))
master_thread.start()

master_thread.join()
for t in client_threads:
    t.join()
    


Exception in thread Thread-21 (run_client):
Traceback (most recent call last):
  File "/home/black_samorez/MiniFL/.conda/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-8 (run_client):
Traceback (most recent call last):
  File "/home/black_samorez/MiniFL/.conda/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-23 (run_client):
Traceback (most recent call last):
  File "/home/black_samorez/MiniFL/.conda/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/black_samorez/MiniFL/.conda/lib/python3.11/threading.py", line 975, in run
Exception in thread Thread-17 (run_client):
Traceback (most recent call last):
  File "/home/black_samorez/MiniFL/.conda/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-4 (run_client):
Traceback (most recent call last):
  File "/home/black_samorez/MiniFL/.conda/lib/python3.11/threading.py", line 1038, in _boot